In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

pd.set_option("max_columns",100)

DATA_DIR="./Data/OnlineNewsPopularity/"

In [7]:
df=pd.read_csv(DATA_DIR+"OnlineNewsPopularity.csv")
meta=pd.read_csv("Articles_meta.csv")
body=pd.read_csv("Articles_body.csv")

In [ ]:
files=pd.read_csv(DATA_DIR+"files.csv")

In [ ]:
files

In [ ]:
print(meta.values[13])
print(body.values[0])
df.iloc[:5,:]

In [ ]:
sentences=meta.values[:5,1]
contents=body.values[:5,1]

In [ ]:
import re
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from textblob import TextBlob

vader=SentimentIntensityAnalyzer()

stopw=set(stopwords.words('english'))
for index,(title_s, content_s) in enumerate(zip(sentences, contents), start=1):
    sentiment_l=[]
    non_stop_w=[]
    wordlen=0
    print(index)
    
    title=re.findall(r'\w+', title_s)
    content=re.findall(r'\w+', content_s)
    for word in content:
        sentiment_l.append(vader.polarity_scores(word))
        sentiment_l[-1].update(dict(TextBlob(word).sentiment._asdict()))
        wordlen+=len(word)
        if word not in stopw:
            non_stop_w.append(word)
    sentiment_df=pd.DataFrame(sentiment_l)
    print(len(title), len(content))
    print(len(np.unique(title)), len(np.unique(content))/len(content))
    non_stop_w=[i for i in content if i not in stopw]
    print(len(non_stop_w), len(np.unique(non_stop_w))/len(non_stop_w))
    wordlen=sum([len(word) for word in content])/len(content)
    print(wordlen)
    print(TextBlob(content_s).sentiment)
    print(TextBlob(title_s).sentiment)
    polarity=
    print(vader.polarity_scores(content_s))
    print(vader.polarity_scores(title_s))
    print()

In [ ]:
files=meta[meta['Title'].isnull()].index.tolist()

In [ ]:
files

In [ ]:
from importlib import reload  
import preprocess
from pandarallel import pandarallel
reload(preprocess)

pandarallel.initialize()

def preproc(row):
    try:
        with open("./Data/OnlineNewsPopularity/mashable/%s"%row.file, "r") as f:
            ret_d={'url':row.url}
            ret_d.update(preprocess.Article(row.url, f.read()).stats())
            return ret_d
    except Exception as e:
        print(e, row.file, row.url)
        return None
df=files.parallel_apply(preproc, axis=1, result_type='expand')

In [ ]:
df.to_csv("df_preprocessed.csv", index=False)

In [8]:
df_new=pd.read_csv("df_preprocessed.csv")

In [ ]:
count=0
for file in files:
    try:
        with open("./Data/OnlineNewsPopularity/mashable/%d.html"%file, "r") as f:
            px=preprocess.Article(meta['url'][1], f.read())
            print(px.stats())
    except Exception as e:
        count+=1
        print(file, e)

In [ ]:
count

In [ ]:
px.metadata

In [ ]:
px.stats()

In [ ]:
from nltk.tokenize import word_tokenize

title=re.findall(r'\w+', sentences[1])
len(title)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from bs4 import BeautifulSoup
f=open("./Data/OnlineNewsPopularity/mashable/11.html", "r")
soup=BeautifulSoup(f.read())
f.close()

In [ ]:
import re
title_l=[ ("meta", {"property": re.compile("title.*")}), ("mega", {"property": re.compile(".*title.*")}), ("meta", {"property": re.compile(".*title.*")})]

for args in title_l:

    title=soup.find(*args)
    if title:
        break

In [ ]:
x=soup.find("section").findAll("a", href=True)
x[0]['href']

In [ ]:
len(soup.find("article").findAll("iframe"))

In [ ]:
len((" ".join(list(soup.find('section').stripped_strings)).split())